<a href="https://colab.research.google.com/github/vishxl/Mask-Mistakes/blob/master/MaskMistakesYOLOv4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Connect google drive**

In [ ]:
!apt update
#!apt upgrade -y
!uname -m && cat /etc/*release
!gcc --version
!uname -r

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done
Building dependency tree       
Reading state information... Done
77 packages can be upgraded. Run 'apt

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Thu Sep 10 17:17:44 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

**1) Clone, configure & compile Darknet**



In [ ]:
# Clone
!git clone https://github.com/AlexeyAB/darknet/
%cd darknet/

In [ ]:
!apt install libopencv-dev python-opencv ffmpeg

In [ ]:
# Configure
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/g' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/g' Makefile

In [ ]:
# Compile
!make

**2) Configure yolov4.cfg file**

In [ ]:
# Make a copy of yolov4.cfg
!cp cfg/yolov4.cfg cfg/yolov4_training.cfg

#Download yolov4.weights file
%cp '../drive/My Drive/Colab Notebooks/Darknet/yolov4.weights' .

In [ ]:
# Change lines in yolov4.cfg file
!sed -i 's/batch=1/batch=64/' cfg/yolov4_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov4_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 6000/' cfg/yolov4_training.cfg
!sed -i '610 s@classes=80@classes=3@' cfg/yolov4_training.cfg
!sed -i '696 s@classes=80@classes=3@' cfg/yolov4_training.cfg
!sed -i '783 s@classes=80@classes=3@' cfg/yolov4_training.cfg
!sed -i '603 s@filters=255@filters=24@' cfg/yolov4_training.cfg
!sed -i '689 s@filters=255@filters=24@' cfg/yolov4_training.cfg
!sed -i '776 s@filters=255@filters=24@' cfg/yolov4_training.cfg

**3) Create .names and .data files**

In [ ]:
!echo -e 'Wearing Mask\n2nd item\n3rd item' > data/obj.names
!echo -e 'classes= 3\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov4' > data/obj.data

**4) Save yolov4_training.cfg and obj.names files in Google drive**

In [ ]:
!cp cfg/yolov4_training.cfg /mydrive/yolov4/yolov4_testing.cfg
!cp data/obj.names /mydrive/yolov4/classes.txt

**5) Create a folder and unzip image dataset**

In [ ]:
!mkdir data/obj
!unzip /mydrive/yolov4/images.zip -d data/obj

**6) Create train.txt file**

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
with open("data/train.txt", "w") as f:
    f.write("\n".join(images_list))

**7) Download pre-trained weights for the convolutional layers file**

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

**8) Start training**

In [ ]:
!./darknet detector train data/obj.data cfg/yolov4_training.cfg darknet53.conv.74 -dont_show
!./darknet detector train data/obj.data cfg/yolov4_training.cfg /mydrive/yolov4/yolov4_training_last.weights -dont_show